In [ ]:
def Ordered_Relationship_Tuple_Constructor(all_relationship_tuples_list):

  relationship_tuples_dict = {}

  for reference_tuple in all_relationship_tuples_list:

    reference_tuple_cell_label = reference_tuple[0][0]

    if reference_tuple_cell_label not in relationship_tuples_dict:
      relationship_tuples_dict[reference_tuple_cell_label] = []

      for other_tuple in all_relationship_tuples_list:

        if other_tuple[0][0] == reference_tuple_cell_label:

          list_of_tuples = relationship_tuples_dict.get(reference_tuple_cell_label)
          list_of_tuples.append(other_tuple)
          relationship_tuples_dict[reference_tuple_cell_label] = list_of_tuples

      tuple_list_with_repeats = relationship_tuples_dict.get(reference_tuple_cell_label)
      list_of_tuples = list(set([rel1_rel2_weight_tuple for rel1_rel2_weight_tuple in tuple_list_with_repeats]))

      list_of_all_dup_indicies = []

      all_duplicate_reference_comparison_index_list = []

      for reference_index, rel1_rel2_weight_tuple_1 in enumerate(list_of_tuples):

        r1_label_reference = rel1_rel2_weight_tuple_1[0][0]
        r2_label_reference = rel1_rel2_weight_tuple_1[1][0]

        r1_2_label_reference_cluster = rel1_rel2_weight_tuple_1[3]

        r1_label_reference_weight = rel1_rel2_weight_tuple_1[0][1]
        r2_label_reference_weight = rel1_rel2_weight_tuple_1[1][1]

        for comparison_index, rel1_rel2_weight_tuple_2 in enumerate(list_of_tuples):

          r1_label_comparison = rel1_rel2_weight_tuple_2[0][0]
          r2_label_comparison = rel1_rel2_weight_tuple_2[1][0]

          r1_2_label_comparison_cluster = rel1_rel2_weight_tuple_2[3]

          r1_label_comparison_weight = rel1_rel2_weight_tuple_2[0][1]
          r2_label_comparison_weight = rel1_rel2_weight_tuple_2[1][1]

          if (r1_label_reference == r1_label_comparison) and (r2_label_reference == r2_label_comparison) and (r1_2_label_reference_cluster != r1_2_label_comparison_cluster) and (r1_label_reference_weight > 10) and (r2_label_reference_weight > 10) and  (r1_label_comparison_weight > 10) and  (r2_label_comparison_weight > 10):

            all_duplicate_reference_comparison_index_list.append((reference_index,comparison_index))
            list_of_all_dup_indicies.append(reference_index)
            list_of_all_dup_indicies.append(comparison_index)


      list_of_tuples_no_dups = Remove_Duplicate_Label_Comparison_Pairs(all_duplicate_reference_comparison_index_list, list_of_tuples, list_of_all_dup_indicies)
      sorted_list = sorted(list_of_tuples_no_dups, key=lambda t: t[2], reverse = True)
      relationship_tuples_dict[reference_tuple_cell_label] = sorted_list


  all_clusters = relationship_tuples_dict.keys()
  # original , key=lambda x: int(x.split('_')[0].split('C')[1])
  all_clusters_sorted = sorted(all_clusters)

  return relationship_tuples_dict, all_clusters_sorted

In [ ]:
def Remove_Duplicate_Label_Comparison_Pairs(all_duplicate_reference_comparison_index_list, list_of_tuples, list_of_all_dup_indicies):

  list_of_tuples_combined = []
  list_of_tuples_no_dups = []

  # Removes all duplicate label comparison pairs and combines them thus two labels of the same labels compaired in different integrated clusters are combined into one comparison percentage
  if all_duplicate_reference_comparison_index_list:
    for dup_ref_comp in all_duplicate_reference_comparison_index_list:

      dup_ref = dup_ref_comp[0]
      dup_comp = dup_ref_comp[1]

      rel1_rel2_weight_tuple_ref = list_of_tuples[dup_ref]
      rel1_rel2_weight_tuple_comp = list_of_tuples[dup_comp]

      r1_label = rel1_rel2_weight_tuple_ref[0][0]
      r2_label = rel1_rel2_weight_tuple_ref[1][0]

      ref_value_1 = rel1_rel2_weight_tuple_ref[0][1]
      ref_value_2 = rel1_rel2_weight_tuple_ref[1][1]
      ref_cluster = rel1_rel2_weight_tuple_ref[3]

      comp_value_1 = rel1_rel2_weight_tuple_comp[0][1]
      comp_value_2 = rel1_rel2_weight_tuple_comp[1][1]
      comp_cluster = rel1_rel2_weight_tuple_comp[3]

      r1_new_total = ref_value_1+comp_value_1
      r2_new_total = ref_value_2+comp_value_2

      rel1_rel2_weight_tuple_combined = ((r1_label, r1_new_total), (r2_label, r2_new_total), r1_new_total+ r2_new_total, str(ref_cluster) + ' ' + str(comp_cluster))

      list_of_tuples_combined.append(rel1_rel2_weight_tuple_combined)

    for index in range(len(list_of_tuples)):
      if index not in list_of_all_dup_indicies:
        list_of_tuples_no_dups.append(list_of_tuples[index])

    for combined_tup in list_of_tuples_combined:
      list_of_tuples_no_dups.append(combined_tup)

  else:
    list_of_tuples_no_dups = list_of_tuples

  return list_of_tuples_no_dups

In [ ]:
def Old_Cluster_Label_and_Counts_Constructor(timepoints_anndata):

  all_timepoints_old_cluster_label_and_counts_dict = {}

  for anndata in timepoints_anndata:
    all_cluster_labels_old = anndata.obs['cluster_labels'].unique().tolist()
    for old_cluster_labels in all_cluster_labels_old:
      all_timepoints_old_cluster_label_and_counts_dict[old_cluster_labels] = anndata.obs.cluster_labels.value_counts()[old_cluster_labels]

  return all_timepoints_old_cluster_label_and_counts_dict

In [ ]:
def Cell_Label_Old_Percentage_Counts_Constructor(all_timepoints_new_cluster_label_and_counts_dict, all_timepoints_old_cluster_label_and_counts_dict):

  cell_label_old_percentage_counts_tuple_list = []

  for cluster_label in all_timepoints_new_cluster_label_and_counts_dict:
    old_counts = all_timepoints_old_cluster_label_and_counts_dict.get(cluster_label)
    new_counts = all_timepoints_new_cluster_label_and_counts_dict.get(cluster_label)
    total_percentage = (new_counts/old_counts)*100
    cluster_label_percentage_tuple = (cluster_label, total_percentage)
    cell_label_old_percentage_counts_tuple_list.append(cluster_label_percentage_tuple)

  return cell_label_old_percentage_counts_tuple_list

In [ ]:
def New_Cluster_Label_and_Counts_Constructor(all_timepoints_integrated, clust_num):

  all_timepoints_new_cluster_label_and_counts_dict = {}

  clust_num_and_clust_label_counts = all_timepoints_integrated.obs[["leiden", "cluster_labels"]].value_counts()[clust_num]

  for new_cluster_labels, new_cluster_label_counts in zip(clust_num_and_clust_label_counts.index, clust_num_and_clust_label_counts):

    all_timepoints_new_cluster_label_and_counts_dict[new_cluster_labels] = new_cluster_label_counts

  return all_timepoints_new_cluster_label_and_counts_dict

In [ ]:
def Relation_Tuples_Constructor(cell_label_old_percentage_counts_tuple_list, clust_num, rel1_rel2_weight_tuple_unique_set_list, rel1_rel2_weight_tuple_list):

  for index in range(len(cell_label_old_percentage_counts_tuple_list)-2):

    reference_label_1 = cell_label_old_percentage_counts_tuple_list[index]

    for cell_label_old_percentage in cell_label_old_percentage_counts_tuple_list[index+1:]:

      weight = reference_label_1[1]+cell_label_old_percentage[1]
      reference_label_2 = cell_label_old_percentage

      if reference_label_1[0].split('_')[1] != reference_label_2[0].split('_')[1]:

        cell_label_old_percentage_relationship_weight_tuple = (reference_label_1, reference_label_2, weight, clust_num)

        cell_label_old_percentage_relationship_weight_tuple_reversed = (reference_label_2, reference_label_1, weight, clust_num)

        rel1_rel2_weight_tuple_unique_set_list.append(cell_label_old_percentage_relationship_weight_tuple)
        rel1_rel2_weight_tuple_list.extend([cell_label_old_percentage_relationship_weight_tuple, cell_label_old_percentage_relationship_weight_tuple_reversed])

  return rel1_rel2_weight_tuple_unique_set_list, rel1_rel2_weight_tuple_list

In [ ]:
def Harmony_Correlations_Text_File_Writer(all_timepoints_integrated_label, relationship_tuples_dict, percentage_correlated_file, all_clusters_sorted):

  open_percentage_correlated_file = open(percentage_correlated_file, "w+")
  open_percentage_correlated_file.write(all_timepoints_integrated_label+' Clusteral Relationship (>10% of cells)\n\n')

  for clusters in all_clusters_sorted:

    list_of_tups = relationship_tuples_dict.get(clusters)

    open_percentage_correlated_file.write(clusters.split('_')[1]+ ' Cluster '+clusters.split('_')[0]+':')
    open_percentage_correlated_file.write('\n\n')

    for x, vals in enumerate(list_of_tups):

      if (vals[0][1] >= 0) and (vals[1][1] >= 0):

        open_percentage_correlated_file.write(vals[0][0]+': '+'{:.2f}'.format(vals[0][1])+'%'+'    <---->    '+vals[1][0]+': '+'{:.2f}'.format(vals[1][1])+'%'+'   '+'Integrated Cluster: '+vals[3])
        open_percentage_correlated_file.write('\n')

      if x == 10:
        break

    open_percentage_correlated_file.write('\n\n')

  open_percentage_correlated_file.close()

In [ ]:
# Harmony Integrated Correlations #
def Harmony_Integrated_Correlations(all_timepoints_integrated_list, all_timepoints_integrated_labels_list, all_timepoints_integrated_percentage_correlated_dir, timepoints_anndata):

  all_timepoints_old_cluster_label_and_counts_dict = Old_Cluster_Label_and_Counts_Constructor(timepoints_anndata)

  all_timepoints_cell_label_old_percentage_counts_tuple_dict = {}
  all_timepoints_cell_label_old_percentage_counts_tuple_unique_set_dict = {}
  all_timepoints_relationship_tuples_dict = {}

  for all_timepoints_integrated_label, all_timepoints_integrated in zip(all_timepoints_integrated_labels_list, all_timepoints_integrated_list):


    all_clusters_new = all_timepoints_integrated.obs['leiden'].unique().tolist()
    rel1_rel2_weight_tuple_list = []
    rel1_rel2_weight_tuple_unique_set_list = []

    for clust_num in all_clusters_new:

      all_timepoints_new_cluster_label_and_counts_dict = New_Cluster_Label_and_Counts_Constructor(all_timepoints_integrated, clust_num)

      cell_label_old_percentage_counts_tuple_list = Cell_Label_Old_Percentage_Counts_Constructor(all_timepoints_new_cluster_label_and_counts_dict, all_timepoints_old_cluster_label_and_counts_dict)

      rel1_rel2_weight_tuple_unique_set_list, rel1_rel2_weight_tuple_list = Relation_Tuples_Constructor(cell_label_old_percentage_counts_tuple_list, clust_num, rel1_rel2_weight_tuple_unique_set_list, rel1_rel2_weight_tuple_unique_set_list)

    all_timepoints_cell_label_old_percentage_counts_tuple_dict[all_timepoints_integrated_label] = rel1_rel2_weight_tuple_list
    all_timepoints_cell_label_old_percentage_counts_tuple_unique_set_dict[all_timepoints_integrated_label] = rel1_rel2_weight_tuple_unique_set_list

  for all_timepoints_integrated_label, percentage_correlated_file in zip(all_timepoints_cell_label_old_percentage_counts_tuple_dict, all_timepoints_integrated_percentage_correlated_dir):

    all_relationship_tuples_list = all_timepoints_cell_label_old_percentage_counts_tuple_dict.get(all_timepoints_integrated_label)

    relationship_tuples_dict, all_clusters_sorted = Ordered_Relationship_Tuple_Constructor(all_relationship_tuples_list)

    Harmony_Correlations_Text_File_Writer(all_timepoints_integrated_label, relationship_tuples_dict, percentage_correlated_file, all_clusters_sorted)

    all_timepoints_relationship_tuples_dict[all_timepoints_integrated_label] = relationship_tuples_dict

  return all_timepoints_relationship_tuples_dict